# BeautyfulSoup 네이버 블로그 스크래핑

## 블로그 포스트 스크래핑 함수 정의

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd

def get_posts(query, page_num=10):

  url_query = quote(query)
  url = "https://search.naver.com/search.naver?where=post&sm=tab_jum&query=" + url_query
  post_df = pd.DataFrame(columns=("Title", "Date", "Blogger", "Blog URL", "Post URL", "Post"))
  idx = 0

  for _ in range(0, page_num):

    search_url = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(search_url, 'html.parser')
    posts = soup.find_all('li', {'class':'sh_blog_top'})

    for post in posts:
      title = post.find('a', {'class':'sh_blog_title _sp_each_url _sp_each_title'}).get('title')
      date = post.find('dd', {'class':'txt_inline'}).get_text()
      blogger = post.find('a', {'class':'txt84'}).get_text()
      blog_url = post.find('a', {'class':'txt84'}).get('href')
      post_url = post.find('a', {'class':'url'}).get('href')
      post_link = urllib.request.urlopen(post_url).read()
      post_html = BeautifulSoup(post_link, 'html.parser')

      post_content_text="" #메인 프레임 여러개일 경우 post_content_text 더해줄 변수

      for main_frame in post_html.select("iframe#mainFrame"): #네이버 블로그에서 메인프레임은 보통 1개이지만, 여러개일 경우 대비
        frame_url = "https://blog.naver.com" + main_frame.get('src')
        post_text = urllib.request.urlopen(frame_url).read()
        post_html = BeautifulSoup(post_text, 'html.parser')
        #post_content_text = ""
          #for post_content in post_html.find_all('div', {'class':'se-main-container'}):
            #post_content_text = post_content.get_text()

        try:
          post_content_text += post_html.find('div', {'class':'se-main-container'}).get_text()
          post_content_text = post_content_text.replace("\n", "")
          post_content_text = post_content_text.replace("\t", "")
        except:
          continue

      post_df.loc[idx] = [title, date, blogger, blog_url, post_url, post_content_text]
      idx += 1
      print("#", end="")

    try:
      next = soup.find('a', {'class':'next'}).get('href')
      url = "https://search.naver.com/search.naver" + next
    except:
      break

  return post_df

## 블로그 포스트 스크래핑 실행

In [ ]:
query = input("검색 질의: ")
post_df = get_posts(query, 10)
print("Done")

검색 질의: 컴퓨터
####################################################################################################Done


In [ ]:
post_df

,Title,Date,Blogger,Blog URL,Post URL,Post
0,니가 왜?? AMD 라이젠5 3600 + RTX2060 신품같은 중고 게임용컴퓨터가...,22시간 전,루나컴퓨터,https://jady06.blog.me,https://jady06.blog.me/222070524365,
1,I7+2060S 조합 고사양게임용컴퓨터,7일 전,원트컴 (구.마블컴),https://blog.naver.com/all_in_box,https://blog.naver.com/all_in_box?Redirect=Log...,"고사양게임용컴퓨터​오늘은 일반적인 게이밍 피씨 옵션중에서도 조금 더, 나은 환경에서..."
2,숯공장에서 온라인주문 확인할 저렴한 컴퓨터. 모니터 포함 30만원이하로.../ 라이...,어제,.,https://blog.naver.com/zxcvnm13k,https://blog.naver.com/zxcvnm13k?Redirect=Log&...,​공장에서 온라인 주문만 하면 된다고 하여... 문의한 분이다. 알고보니 대략 8년...
3,4K 작업에 최적화된 영상편집용 컴퓨터,3일 전,조인스컴 공식블로그,https://blog.naver.com/hanwoo10303,https://blog.naver.com/hanwoo10303?Redirect=Lo...,오늘은 성능과 가성비를 동시에 잡은 4K 영상편집용 컴퓨터를 소개해 드리겠습니다. ...
4,강남컴퓨터학원 꿈을 위한 한걸음!,20시간 전,English and Life♥,https://blog.naver.com/aldus9834,https://blog.naver.com/aldus9834?Redirect=Log&...,여태 다루고 있었던 전공과는무관하지만 그래도 배우고 싶었던게 있었어요혼자 시작하려 ...
...,...,...,...,...,...,...
95,컴퓨터가 느려진다면? 구라제거기 다운로드 설치 방법은?,2020.03.31.,영댕이의 funfun한 ...,https://funfunhan.com/,https://funfunhan.com/2476774?category=158981,
96,컴퓨터와 키보드 청소를 위한 알리발 먼지제거기 언박싱!,2020.04.14.,영댕이의 funfun한 ...,https://funfunhan.com/,https://funfunhan.com/2476779?category=41829,
97,컴퓨터 화면 캡쳐 기본 프로그램과 단축키 이용하는 방법,2020.04.16.,모이모이의 IT 이야...,https://blog.naver.com/moimoi1357,https://blog.naver.com/moimoi1357?Redirect=Log...,다양한 최신 IT 기기에 대해 살펴보고 성능과 장단점에 대한 테스트를 진행하면서 사...
98,"화이트 맞춤 인텔 i7 10700F 조합으로 게임방송,3D 영상편집용 멀티플레이어 ...",2020.07.29.,연우컴퓨터 1.1,https://blog.naver.com/sham0089,https://blog.naver.com/sham0089?Redirect=Log&l...,안녕하세요 7월 한 달 내내 비 소식으로 이제는 비가 그만 내리면 좋겠다고비가 너...
